<a href="https://colab.research.google.com/github/talhaanwarch/skin-disease-diagnosis/blob/master/Scenario_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install efficientnet

In [2]:
#download the dataset
!wget https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/zr7vgbcyr2-1.zip

--2020-09-27 04:33:38--  https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/zr7vgbcyr2-1.zip
Resolving md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com (md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com)... 52.218.88.136
Connecting to md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com (md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com)|52.218.88.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3593344681 (3.3G) [application/octet-stream]
Saving to: ‘zr7vgbcyr2-1.zip’

zr7vgbcyr2-1.zip    100%[===================>]   3.35G  21.0MB/s    in 2m 41s  

2020-09-27 04:36:20 (21.3 MB/s) - ‘zr7vgbcyr2-1.zip’ saved [3593344681/3593344681]



In [3]:
#unzip the dataset
%%capture
!unzip zr7vgbcyr2-1.zip

In [4]:
#create a new folder named as data
!mkdir data

In [5]:
#unzip all subfolder
%%capture
!unzip /content/images/imgs_part_1.zip
!unzip /content/images/imgs_part_2.zip
!unzip /content/images/imgs_part_3.zip

In [6]:
#move all pictures from subfolder to main folder data
!mv  /content/imgs_part_1/* /content/data
!mv  /content/imgs_part_2/* /content/data
!mv  /content/imgs_part_3/* /content/data

In [7]:
#remove the folder that contain subfolders
!rm -r /content/images

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#read annotation file
df=pd.read_csv('/content/metadata.csv')
df.head()

,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,cancer_history,has_piped_water,has_sewage_system,fitspatrick,region,diameter_1,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
0,PAT_1516,1765,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARM,NaN,NaN,NEV,False,False,False,False,False,False,PAT_1516_1765_530.png,False
1,PAT_46,881,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,True,True,True,3.0,NECK,6.0,5.0,BCC,True,True,False,True,True,True,PAT_46_881_939.png,True
2,PAT_1545,1867,NaN,NaN,NaN,NaN,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FACE,NaN,NaN,ACK,True,False,False,False,False,False,PAT_1545_1867_547.png,False
3,PAT_1989,4061,NaN,NaN,NaN,NaN,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HAND,NaN,NaN,ACK,True,False,False,False,False,False,PAT_1989_4061_934.png,False
4,PAT_684,1302,False,True,POMERANIA,POMERANIA,79,False,MALE,True,False,False,False,1.0,FOREARM,5.0,5.0,BCC,True,True,False,False,True,True,PAT_684_1302_588.png,True


In [9]:
#shuffle the dataframe
df=df.sample(frac=1)
df.head()

,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,cancer_history,has_piped_water,has_sewage_system,fitspatrick,region,diameter_1,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
1255,PAT_1118,470,NaN,NaN,NaN,NaN,55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FACE,NaN,NaN,ACK,True,True,False,False,False,False,PAT_1118_470_710.png,False
1987,PAT_641,3704,False,False,GERMANY,NaN,20,False,FEMALE,True,False,True,False,2.0,ABDOMEN,5.0,3.0,NEV,False,False,False,False,False,False,PAT_641_3704_317.png,False
1665,PAT_433,2650,False,True,GERMANY,GERMANY,30,False,MALE,False,True,False,False,2.0,NECK,20.0,20.0,NEV,False,True,False,False,False,True,PAT_433_2650_417.png,False
1107,PAT_104,1754,False,False,POMERANIA,POMERANIA,79,True,FEMALE,True,True,False,False,2.0,ARM,11.0,10.0,ACK,True,False,False,False,False,True,PAT_104_1754_276.png,True
1916,PAT_656,1246,False,False,UNK,UNK,58,False,MALE,False,False,True,True,2.0,BACK,18.0,12.0,MEL,True,True,False,True,False,False,PAT_656_1246_489.png,True


In [10]:
#create a dataframe with image name and encoded labels
labels=pd.get_dummies(df['diagnostic'])
df_img=pd.concat([df['img_id'],labels],axis=1)
df_img.head()
#MEL	NV	BCC	AK	BKL		SCC

,img_id,ACK,BCC,MEL,NEV,SCC,SEK
1255,PAT_1118_470_710.png,1,0,0,0,0,0
1987,PAT_641_3704_317.png,0,0,0,1,0,0
1665,PAT_433_2650_417.png,0,0,0,1,0,0
1107,PAT_104_1754_276.png,1,0,0,0,0,0
1916,PAT_656_1246_489.png,0,0,1,0,0,0


In [11]:
#labels columns in dataframe
col=list(df_img.columns)[1::]
col

['ACK', 'BCC', 'MEL', 'NEV', 'SCC', 'SEK']

In [12]:
df['smoke']=df['smoke'].fillna('unknown').astype('str')
df['drink']=df['drink'].fillna('unknown').astype('str')
df['pesticide']=df['pesticide'].fillna('unknown').astype('str')
df['gender']=df['gender'].fillna('unknown').astype('str')
df['skin_cancer_history']=df['skin_cancer_history'].fillna('unknown').astype('str')
df['cancer_history']=df['cancer_history'].fillna('unknown').astype('str')
df['has_piped_water']=df['has_piped_water'].fillna('unknown').astype('str')
df['has_sewage_system']=df['has_sewage_system'].fillna('unknown').astype('str')

df['fitspatrick']=df['fitspatrick'].fillna('unknown').astype('str')
df['has_sewage_system']=df['has_sewage_system'].fillna('unknown').astype('str')
df['diameter_1'].fillna(df['diameter_1'].mean(),inplace=True)
df['diameter_2'].fillna(df['diameter_2'].mean(),inplace=True)

In [13]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
encLE=LabelEncoder()
encOHE=OneHotEncoder(sparse=False)

In [14]:
for i in df.columns[2::]:
  if i!='diagnostic' or i!='img_id':
    df[i]=encLE.fit_transform(df[i].astype('str'))

In [15]:
tab_features=df.drop(['patient_id','lesion_id','diagnostic','img_id'],axis=1)
tab_labels=df['diagnostic']
tab_features.head()

,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,cancer_history,has_piped_water,has_sewage_system,fitspatrick,region,diameter_1,diameter_2,itch,grew,hurt,changed,bleed,elevation,biopsed
1255,2,2,13,11,42,2,2,2,2,2,2,6,5,6,37,1,1,0,0,0,0,0
1987,0,0,4,11,7,0,0,1,0,1,0,1,0,33,23,0,0,0,0,0,0,0
1665,0,1,4,2,17,0,1,0,1,0,0,1,10,16,14,0,1,0,0,0,1,0
1107,0,0,9,7,67,1,0,1,1,0,0,1,1,5,3,1,0,0,0,0,1,1
1916,0,0,12,10,45,0,1,0,0,1,1,1,2,13,5,1,1,0,1,0,0,1


In [16]:
# data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
data_gen= ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=360,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.1,
    rescale=1/255,
    fill_mode='nearest')

In [17]:
# model hyperparameters
img_shape=300
batch_size=20
n_epochs=20

In [18]:
from tensorflow.keras.layers import GlobalAveragePooling2D,Dropout,Dense
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential,Model

In [19]:
from xgboost import XGBClassifier
xgb_clf = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=1,
              learning_rate=0.5, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=1, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=True, subsample=1.0, verbosity=1)

In [20]:
def basemodel():
  import efficientnet.tfkeras as efn
  base_model =efn.EfficientNetB3(weights ='noisy-student', include_top=False, input_shape = (img_shape,img_shape,3))
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dropout(0.3)(x)
  x = Dense(128, activation="relu")(x)
  x = Dropout(0.3)(x)
  x = Dense(64, activation="relu")(x)
  predictions = Dense(len(col), activation="softmax")(x) 
  base_model = Model(inputs=base_model.input, outputs=predictions)
  base_model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])
  return base_model

In [ ]:

from sklearn.model_selection import KFold,StratifiedKFold
import gc

from sklearn.metrics import classification_report,accuracy_score,f1_score,roc_auc_score,recall_score,precision_score,balanced_accuracy_score
classification_reports=[]
kf = KFold(n_splits=5,shuffle=True)
history=[]
accuracy=[]
f1_scores=[]
auc=[]
y_pred=[]
precision=[]
recall=[]
balanced_acc=[]
fold=0
for train_index, test_index in kf.split(df_img):
  train=df_img.iloc[train_index,:]
  val=df_img.iloc[test_index,:]
  print('-----------fold {}--------------'.format(fold))
  train_generator=data_gen.flow_from_dataframe(train,directory='/content/data',
                                               target_size=(img_shape,img_shape),
                                               x_col='img_id',
                                               y_col=col,
                                               class_mode='raw',
                                               shuffle=True,
                                               batch_size=batch_size)

  val_generator=data_gen.flow_from_dataframe(val,directory='/content/data',
                                             target_size=(img_shape,img_shape),
                                             x_col="img_id",
                                             y_col=col,
                                             class_mode='raw',
                                             shuffle=False,
                                             batch_size=batch_size)
  #train_generator.next()[0].shape,train_generator.next()[1].shape

  #load base model
  base_model=basemodel()
  #train model
  results = base_model.fit(train_generator,epochs=25,verbose=0,
                              steps_per_epoch=train_generator.n/batch_size,
                              validation_data=val_generator,
                             validation_steps=val_generator.n/batch_size,
                              callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=5, min_lr=0.000001),
                                         #ModelCheckpoint('model_{}.hdf5'.format(fold), save_best_only=True, monitor='val_loss', mode='min')
                                         ])
  y_true=val.iloc[:,1::]
  val_generator.reset()
  img_pred=base_model.predict(val_generator,steps=val_generator.n/batch_size,verbose=0)

  xgb_clf.fit(tab_features.values[train_index,:],tab_labels.iloc[train_index],verbose=False)
  tab_pred=xgb_clf.predict_proba(tab_features.values[test_index,:])

  tab_f1=f1_score(y_true,tab_pred.round().astype(int),average='macro',labels=np.unique(tab_pred))
  img_f1=f1_score(y_true,img_pred.round().astype(int),average='macro',labels=np.unique(img_pred))
  print('f1 score is ', img_f1,' and ',tab_f1)

  wt=tab_f1+img_f1
  y_pred=((tab_f1/wt)*tab_pred)+((img_f1/wt)*img_pred)
  y_pred=np.round(y_pred,0)
  accuracy.append(accuracy_score(y_true,y_pred))
  auc.append(roc_auc_score(y_true,y_pred))
  precision.append(precision_score(y_true,y_pred,average='macro',labels=np.unique(y_pred)))
  recall.append(recall_score(y_true,y_pred,average='macro',labels=np.unique(y_pred)))
  f1_scores.append(f1_score(y_true,y_pred,average='macro',labels=np.unique(y_pred)))
  balanced_acc.append(balanced_accuracy_score(y_true.values.argmax(axis=1), y_pred.argmax(axis=1)))

  history.append(results)
  fold+=1

  del base_model
  K.clear_session()
  gc.collect()

-----------fold 0--------------
Found 1838 validated image filenames.
Found 460 validated image filenames.
43933696/43933088 [==============================] - 1s 0us/step
f1 score is  0.7947019867549667  and  0.8438538205980067
-----------fold 1--------------
Found 1838 validated image filenames.
Found 460 validated image filenames.


In [ ]:
#average accuracy across all folds
np.mean(accuracy)

In [ ]:
#average precision across all folds
np.mean(precision)

In [ ]:
#average recall across all folds
np.mean(recall)

In [ ]:
#average f1_scores across all folds
np.mean(f1_scores)

In [ ]:
#average auc_scores across all folds
np.mean(auc)

In [ ]:
np.mean(balanced_acc)

In [ ]:
history[0].history.keys()

In [ ]:
def plot_accuracy(hist,acc='val_accuracy'):
    for i in range(5):
      plt.plot(hist[i].history[acc],linestyle='-.',linewidth=1.5)
     
    mean=np.mean([[j for j in x.history[acc]] for x in hist],axis=0)
    std=np.std([[j for j in x.history[acc]] for x in hist],axis=0)
    plt.plot(mean,linewidth=2,color='black')
    if acc=='val_accuracy':
      plt.title('Model Validation Accuracy')
    else:
      plt.title('Model Training Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epochs')
    plt.legend(['Fold1','Fold2','Fold3','Fold4','Fold5','Average Val Accuracy' ], loc='best')
    plt.show()

In [ ]:
plot_accuracy(history)

In [ ]:
plot_accuracy(history,acc='accuracy')